In [9]:
import pandas as pd

df = pd.read_csv("ecommerce_dataset_updated.csv")

# Convert date
df['Purchase_Date'] = pd.to_datetime(df['Purchase_Date'])


/var/folders/tn/nx0fp4655xs3pdpnrh5z4yt80000gn/T/ipykernel_5408/3751567477.py:6: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Purchase_Date'] = pd.to_datetime(df['Purchase_Date'])


In [10]:
df['year'] = df['Purchase_Date'].dt.year
df['month'] = df['Purchase_Date'].dt.month
df['month_name'] = df['Purchase_Date'].dt.month_name()


In [11]:
monthly_revenue = df.groupby(['year', 'month', 'month_name']).agg(
    total_revenue=('net_revenue', 'sum'),
    total_orders=('order_id', 'nunique')
).reset_index()

monthly_revenue = monthly_revenue.sort_values(['year', 'month'])


In [12]:
monthly_revenue['previous_month_revenue'] = monthly_revenue['total_revenue'].shift(1)

monthly_revenue['mom_growth_percent'] = (
    (monthly_revenue['total_revenue'] - monthly_revenue['previous_month_revenue']) /
    monthly_revenue['previous_month_revenue']
) * 100


In [13]:
seasonality = df.groupby('month_name').agg(
    avg_monthly_revenue=('net_revenue', 'sum'),
    avg_orders=('order_id', 'nunique')
).reset_index()

# Order months correctly
month_order = [
    'January','February','March','April','May','June',
    'July','August','September','October','November','December'
]

seasonality['month_name'] = pd.Categorical(
    seasonality['month_name'],
    categories=month_order,
    ordered=True
)

seasonality = seasonality.sort_values('month_name')


In [14]:
monthly_revenue.to_csv("monthly_revenue_trend.csv", index=False)
seasonality.to_csv("revenue_seasonality.csv", index=False)

print("Files saved:")
print("- monthly_revenue_trend.csv")
print("- revenue_seasonality.csv")


Files saved:
- monthly_revenue_trend.csv
- revenue_seasonality.csv
